# Standalone Eval

## 1. Here save the gt in compact format

in original scale!

In [1]:
import sys, os, os.path as osp

sys.path.append(osp.dirname(os.getcwd()))
from matplotlib.axes._axes import _log as matplotlib_axes_logger

matplotlib_axes_logger.setLevel("ERROR")
import yaml, numpy as np, trimesh, imageio
from dataset import get_dataset
from init import setup_seed
from tqdm import tqdm
import pickle

from object_utils.arti_graph_utils_v3 import get_G_from_VE
from object_utils.arti_viz_utils import append_mesh_to_G, viz_G_topology, viz_G
import logging

setup_seed(12345)

logger = logging.getLogger("trimesh")
logger.setLevel(logging.ERROR)


In [2]:
cfg_fn = "../configs/nap/v6.1_diffusion.yaml"
with open(cfg_fn, "r") as f:
    cfg = yaml.full_load(f)
cfg["root"] = osp.dirname(os.getcwd())
cfg["modes"] = ["train", "val", "test"]
cfg["dataset"]["dataset_proportion"] = [1.0, 1.0, 1.0]
# cfg["dataset"]["data_root"] = osp.join("..", cfg["dataset"]["data_root"])
cfg["dataset"]["split_path"] = osp.join("..", cfg["dataset"]["split_path"])
cfg["dataset"]["embedding_index_file"] = osp.join("..", cfg["dataset"]["embedding_index_file"])
cfg["dataset"]["embedding_precompute_path"] = osp.join("..", cfg["dataset"]["embedding_precompute_path"])

cates = cfg["dataset"]["cates"]
K = cfg["dataset"]["max_K"]

In [3]:
dataclass = get_dataset(cfg)
test_dataset = dataclass(cfg, "test")
print(f"K={K} cate={cates} test dataset has {len(test_dataset)} instances")

100%|██████████| 2340/2340 [00:02<00:00, 964.81it/s] 

K=8 cate=['all'] test dataset has 449 instances


In [4]:
DST = f"../log/test/G/K_{K}_cate_{''.join(cates)}_gt"
DST_viz = f"../log/test/Viz/K_{K}_cate_{''.join(cates)}_gt"
GT_MESH_ROOT =  "../data/partnet_mobility_graph_mesh"
os.makedirs(DST, exist_ok=True)
os.makedirs(DST_viz, exist_ok=True)
print(DST)

../log/test/G/K_8_cate_all_gt


In [5]:
def save(p):
    data, meta, dst = p
    fn = osp.join(dst, f"{meta['partnet-m-id']}.pkl")
    mesh_name_list = meta["mesh_name_list"]
    gt_mesh_list = [
        trimesh.load(osp.join(GT_MESH_ROOT, mesh_name + ".off"), force="mesh")
        for mesh_name in mesh_name_list
        if len(mesh_name) > 0
    ]

    V, E = data["V"], data["E"]
    # ! warning, here should multiply back to the original scale, the returned is scaled
    V_scale, E_scale = data["V_scale"], data["E_scale"]
    V = V * V_scale
    E = E * E_scale
    v_mask = V[:, 0] > 0
    assert v_mask.sum() == len(gt_mesh_list)
    assert v_mask[: len(gt_mesh_list)].all()

    G = get_G_from_VE(V.cpu().numpy(), E.cpu().numpy())
    G = append_mesh_to_G(G, gt_mesh_list, key="mesh") # with key mesh

    # viz_topo = viz_G_topology(G)
    # imageio.imwrite(osp.join(DST_viz, f"{meta['partnet-m-id']}.png"), viz_topo)
    # viz_list = viz_G(G, cam_dist=3.0, viz_frame_N=4, shape=(128, 128))
    # imageio.mimsave(osp.join(DST_viz, f"{meta['partnet-m-id']}.gif"), viz_list, fps=10)
    
    pickle.dump(G, open(fn, "wb"))
    data = pickle.load(open(fn, "rb"))
    return


In [6]:
p_list = []
for data, meta in tqdm(test_dataset):
    # save((data, meta))
    p_list.append((data, meta, DST))

from multiprocessing import Pool
with Pool(14) as p:
    p.map(save, p_list)

100%|██████████| 449/449 [00:00<00:00, 985.92it/s] 


In [7]:
from save_viz_utils import viz_dir

viz_dir(DST, DST_viz, max_viz=40, n_threads=4, n_frames=7)

start rendering 41 files...
finished render ../log/test/Viz/K_8_cate_all_gt/48452.gif
finished render ../log/test/Viz/K_8_cate_all_gt/101458.gif
finished save ../log/test/Viz/K_8_cate_all_gt/48452.gif
finished render ../log/test/Viz/K_8_cate_all_gt/10305.gif
finished save ../log/test/Viz/K_8_cate_all_gt/101458.gif
finished save ../log/test/Viz/K_8_cate_all_gt/10305.gif
finished render ../log/test/Viz/K_8_cate_all_gt/102044.gif
finished save ../log/test/Viz/K_8_cate_all_gt/102044.gif
finished render ../log/test/Viz/K_8_cate_all_gt/102915.gif
finished save ../log/test/Viz/K_8_cate_all_gt/102915.gif
finished render ../log/test/Viz/K_8_cate_all_gt/102253.giffinished render ../log/test/Viz/K_8_cate_all_gt/10889.gif

finished save ../log/test/Viz/K_8_cate_all_gt/102253.gif
finished save ../log/test/Viz/K_8_cate_all_gt/10889.gif
finished render ../log/test/Viz/K_8_cate_all_gt/102272.gif
finished save ../log/test/Viz/K_8_cate_all_gt/102272.gif
finished render ../log/test/Viz/K_8_cate_all_gt/10

In [8]:
G = pickle.load(open(osp.join(DST, "100017.pkl"), "rb"))
viz_topo = viz_G_topology(G)
imageio.imwrite(osp.join("debug.png"), viz_topo)
viz_list = viz_G(G, cam_dist=3.0, viz_frame_N=4, shape=(128, 128))
imageio.mimsave(osp.join("debug.gif"), viz_list, fps=10)

/home/ray/anaconda3/envs/nap-gcc9/lib/python3.9/site-packages/networkx/drawing/nx_pylab.py:433: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  node_collection = ax.scatter(
